In [2]:
import re
import numpy as np


data = """
Button A: X+94, Y+34
Button B: X+22, Y+67
Prize: X=8400, Y=5400

Button A: X+26, Y+66
Button B: X+67, Y+21
Prize: X=12748, Y=12176

Button A: X+17, Y+86
Button B: X+84, Y+37
Prize: X=7870, Y=6450

Button A: X+69, Y+23
Button B: X+27, Y+71
Prize: X=18641, Y=10279"""

lines = data.strip().split("\n")
button_a = [tuple(map(int, re.findall(r'\d+', s))) for s in lines[::4]]
button_b = [tuple(map(int, re.findall(r'\d+', s))) for s in lines[1::4]]
prizes = [tuple(map(int, re.findall(r'\d+', s))) for s in lines[2::4]]

prizes = [(10000000000000 + p[0], 10000000000000 + p[1]) for p in prizes]

print(button_a,  button_b, prizes, sep="\n")

[(94, 34), (26, 66), (17, 86), (69, 23)]
[(22, 67), (67, 21), (84, 37), (27, 71)]
[(10000000008400, 10000000005400), (10000000012748, 10000000012176), (10000000007870, 10000000006450), (10000000018641, 10000000010279)]


In [20]:
n = 0
a = button_a[n]
b = button_b[n]
p = prizes[n]

m = np.array([a, b]).T
m, p

(array([[94, 22],
        [34, 67]]),
 (10000000008400, 10000000005400))

In [21]:
r = np.linalg.solve(m, p)

r[0], r[1]

(81081081161.08107, 108108108148.10812)

In [22]:
z = np.round(r, decimals=0)

z[0], z[1]

(81081081161.0, 108108108148.0)

In [23]:
t = np.round(r, decimals=3)

t[0], t[1]

(81081081161.081, 108108108148.108)

In [24]:
np.array_equal(z, t)

False